#Arbol de Decisión
Modelo de clasificación


In [2]:
#Importamos lo necesario: numpy pandas
import numpy as np
import pandas as pd
#Extras
import seaborn as sns
import random
from pprint import pprint


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Cargamos los datos de iris a partir de un csv
datosIris = pd.read_csv("https://raw.githubusercontent.com/adrishg/PythonML-PrebecariosProteco/master/DataSets/iris.csv")
datosIris = datosIris.drop("Id", axis=1)
encabezadosDeColumnas =datosIris.columns.tolist()
datosIris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Los árboles de decisión usan el algoritmo CART (Classification and Regression Tree) para poder hacer una regresión o una clasificación. 

Este algoritmo consiste en separar los datos en dos usando la mejor combinatoria de caracteristica *k* y valor númerico o umbral *t* que separe a los dos subárboles con mejor pureza (mejor gini y menor entropía)




**Requerimientos**

Para poder tener nuestro árbol de clasificación se considera que necesitaremos las siguientes funciones:

  1) Cargar los datos

  2) Función separe training set de test set

  3) FuncionSepareLosDatosEn2

  4) FuncionHagaCombinatoriasDeKyT

  5) FuncionEvalueElGiniOPurezaOEntropía

  6) ÁrbolDeDesición ← Función recursiva que ejecutará todas las funciones anteriores y generará dos subarboles a los cuales se les volverá a aplicar la misma función.
  



In [0]:
#Creamos una función que nos separe los datos completos en dos 
#subsets uno para entrenamiento y uno para test
def separacionDatosTesteo(datosIris, tamanioTest=0.2):
  if isinstance(tamanioTest, float):
    tamanioTest = round(tamanioTest * len(datosIris))

  indices = datosIris.index.tolist()
  testIndices = random.sample(population=indices, k=tamanioTest)

  datosTesteo = datosIris.loc[testIndices]
  datosEntrenamiento = datosIris.drop(testIndices)

  return datosEntrenamiento, datosTesteo

In [0]:
#Separamos nuestros datos con la función declarada arriba 
datosEntrenamiento, datosTesteo = separacionDatosTesteo(datosIris)

In [6]:
#Definimos data como los valores de los datos de Entrenamiento
data= datosEntrenamiento.values
data

array([[4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
       [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
       [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
       [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
       [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
       [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
       [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
       [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
       [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
       [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
       [5.1, 3.3, 1.7, 0.5, 'Iris-setosa'],
       [4.8, 3.4, 1.9, 0.2, 'Iris-setosa'],
       [5.0, 3.0, 1.6, 0.2, 'Iris-setosa'],
       [5.2, 3.5, 1.5, 0.2, 'Iris-setosa'],
       [5.2, 3.4, 1.4, 0.2, 'Iri

Friendly reminder: Entropía o gini = 0 implica pureza.



In [7]:
#Hacemos una función que nos verifique y regrese un True si nuestros datos están puros
def calcularPureza(datos):
  etiquetas=datos[:,-1] #Recuperamos etiquetas del dataset
  etiquetasUnicas = np.unique(etiquetas) #Obtenemos las etiquetas únicas que hay
  if len(etiquetasUnicas)==1:  #Sí sólo hay una etiqueta (es puro)
    return True
  else:
    return False

calcularPureza(data) #Calculamos la pureza inicial de todos los datos (Falso porque son 3 etiquetas)

False

In [0]:
 #Obtiene las etiquetas que hay en el datset, una ventaja de los árboles de decisión es que no sólo clasifican dos grupos como en la regresión logística
def clasificarDatos(data):
  etiquetas=data[:,-1] 
  etiquetasUnicas, cuentaDeEtiquetasUnicas= np.unique(etiquetas, return_counts=True)

  index= cuentaDeEtiquetasUnicas.argmax()
  clasificacion = etiquetasUnicas[index]

  return clasificacion

In [0]:
#Las medidas de ancho y largo de pétalos/sépalos es un valor continuo entonteces para encontrar los valores t posibles 
#usaremos la función Separaciones posibles que toma los valores  los ordena de mayot a menor tomando en cuentas las etiquetas y promedialos valores
def separacionesPosibles(data):
  separaciones= {}
  x, numeroDeColumnas= data.shape

  for i in range(numeroDeColumnas -1):
    separaciones[i] = []
    valores = data[:,i]
    valoresUnicos = np.unique(valores)

    for indice in range(len(valoresUnicos)):
      if indice !=  0:
        valorActual= valoresUnicos[indice]
        valorAnterior =valoresUnicos[indice-1]
        separacionPosible =(valorActual + valorAnterior)/2

        separaciones[i].append(separacionPosible)
  return separaciones

#separacionesPosibles(data)


In [0]:
#Esta función separa los datos
def separarDatos(data, separadorColumna, separadorValor):
  separarValoresColumnas= data[:,separadorColumna]
  datosAbajo= data[separarValoresColumnas <= separadorValor]
  datosArriba= data[separarValoresColumnas> separadorValor]

  return datosAbajo, datosArriba

In [0]:
#En este caso usaremos la entropía como metrica, es inversa a la pureza entonces queremos minimizarla
def calcularEntropia(datos):
  etiquetas=datos[:,-1]
  x, cuenta= np.unique(etiquetas, return_counts=True)
  probabilidad= cuenta/cuenta.sum()
  entropia= sum(probabilidad* -np.log2(probabilidad))

  return entropia

In [0]:
def calcularEntropiaTotal(datosArriba, datosAbajo):
  n= len(datosArriba)+len(datosAbajo)
  proporcionArriba= len(datosArriba)/n
  proporcionAbajo= len(datosAbajo)/n

  entropiaTotal= (proporcionAbajo*calcularEntropia(datosAbajo)+ proporcionArriba* calcularEntropia(datosArriba))
  return entropiaTotal

In [0]:
#Para minimizar la entropía evaluamos que caracteristica k y cuál umbral t nos dan la menor entropía
def mejorSeparacion(data, separaciones):
  #print(np.array([np.array(i) for i in separaciones.values()]))
  mejorSeparacion, mejorColumna= 0,0
  entropia= calcularEntropiaTotal(*separarDatos(data, separadorColumna=0, separadorValor=0))
  for i in separaciones:
    for valor in separaciones[i]:
      datosAbajo, datosArriba = separarDatos(data, separadorColumna=i, separadorValor=valor)
      entropiaActualTotal= calcularEntropiaTotal(datosArriba, datosAbajo)
      if entropiaActualTotal < entropia:
        entropia= entropiaActualTotal
        mejorColumna= i
        mejorValorDeSeparacion= valor
  return mejorColumna, mejorValorDeSeparacion

In [0]:
#Finalmente llegamos a la función de arbolDeDecision. 
#Piensen en esto como su programa main qué manda llamar a las demás funciones 
#Recuerden que será recursiva porque dentro de ella se manda llamar a si misma para los dos subarboles generados
def arbolDeDecision(data, contador=0, minMuestras=2, profundidadMaxima=4):
  datosIris= data
  #Para la recursividad necesitamos un caso base en este caso tomamos en cuenta 3 posible
  #1)Si la función pureza regresa un True
  #2) Sí el numero de muestras es menor a mínimo de muestras (Por default 2)
  #3) Que la profundidad del árbol llegue a la profundidad máxima (Por default 4)
  if(calcularPureza(data)) or (len(data)< minMuestras) or (contador== profundidadMaxima):
    clasificacion= clasificarDatos(data)
    return clasificacion
  else:
    contador +=1

    separaciones = separacionesPosibles(datosIris)
    mejorColumna , mejorValorDeSeparacion = mejorSeparacion(datosIris, separaciones)
    datosAbajo, datosArriba = separarDatos(datosIris, mejorColumna, mejorValorDeSeparacion)
    
    #El árbol estará representado por un diccionario anidado.
    pregunta ="{} <= {}".format(encabezadosDeColumnas[mejorColumna],mejorValorDeSeparacion)
    subArbol={ pregunta: []}

    si = arbolDeDecision(datosAbajo, contador,minMuestras, profundidadMaxima)
    no = arbolDeDecision(datosArriba, contador, minMuestras, profundidadMaxima)

    if si==no:
      subArbol= si
    else:
      subArbol[pregunta].append(si)
      subArbol[pregunta].append(no)
    return subArbol

In [15]:
arbol = arbolDeDecision(data)
pprint(arbol)

{'petal_length <= 2.45': ['Iris-setosa',
                          {'petal_length <= 4.75': [{'sepal_length <= 4.95': ['Iris-virginica',
                                                                              'Iris-versicolor']},
                                                    {'petal_length <= 5.05': [{'sepal_length <= 6.5': ['Iris-virginica',
                                                                                                       'Iris-versicolor']},
                                                                              'Iris-virginica']}]}]}


In [16]:
#Tomamos uno de los datos de testeo
ejemplo = datosTesteo.iloc[0]
ejemplo

sepal_length               6.1
sepal_width                2.6
petal_length               5.6
petal_width                1.4
species         Iris-virginica
Name: 134, dtype: object

In [17]:
ejemplo

sepal_length               6.1
sepal_width                2.6
petal_length               5.6
petal_width                1.4
species         Iris-virginica
Name: 134, dtype: object

In [18]:
#Los datos lo pasamos a un diccionario
ejemplito= {'sepal_length': ejemplo[0], 'sepal_width': ejemplo[1], 
            'petal_length': ejemplo[2],'petal_width': ejemplo[3], 
            'species': ejemplo[4]}
print(ejemplito)

{'sepal_length': 6.1, 'sepal_width': 2.6, 'petal_length': 5.6, 'petal_width': 1.4, 'species': 'Iris-virginica'}


In [19]:
#Ahora intentaremos hacer que una predicción a partir de nuestro modelo.
def ejemploClasificacion(ejemplo, arbol):
  pregunta= list(arbol.keys())[0]
  nombreCaracteristica, operador, valor = pregunta.split(" ")
  if ejemplo[nombreCaracteristica] <= float(valor):
    respuesta= arbol[pregunta][0]
  else:
    respuesta= arbol[pregunta][1]

  if not isinstance(respuesta, dict):
    return respuesta
  else:
    restoDelArbol = respuesta
    return ejemploClasificacion(ejemplo, restoDelArbol)

ejemploClasificacion(ejemplito, arbol)

'Iris-virginica'